In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from pathlib import Path
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir(Path("../input")))
print(os.listdir(Path("../input/embeddings")))


['embeddings', 'sample_submission.csv', 'test.csv', 'train.csv']
['glove.840B.300d', 'GoogleNews-vectors-negative300', 'paragram_300_sl999', 'wiki-news-300d-1M']


In [4]:
#load the data:
train = pd.read_csv(Path("../input/train.csv"))
test = pd.read_csv(Path('../input/test.csv'))
sub = pd.read_csv(Path('../input/sample_submission.csv'))
y = train.target.values
print("done reading input files")
print(test.head())


done reading input files
                    qid                                      question_text
0  00014894849d00ba98a9  My voice range is A2-C5. My chest voice goes u...
1  000156468431f09b3cae           How much does a tutor earn in Bangalore?
2  000227734433360e1aae  What are the best made pocket knives under $20...
3  0005e06fbe3045bd2a92  Why would they add a hypothetical scenario tha...
4  00068a0f7f41f50fc399   What is the dresscode for Techmahindra freshers?


In [5]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train['question_text'], train['target'])

#train_x = train['question_text']
#train_y = train['target']

test_x = test['question_text']

In [9]:
# load the pre-trained word-embedding vectors 
# needs about 999995it to finish
embeddings_index = {}
for i, line in enumerate(open('../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

In [10]:
# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(train['question_text'])
word_index = token.word_index

In [11]:
# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)
test_seq_x = sequence.pad_sequences(token.texts_to_sequences(test_x))

In [13]:

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [14]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

In [15]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [22]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
from tensorflow import keras
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [19]:
classifier = create_cnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("CNN, Word Embeddings",  accuracy)

Epoch 1/1
    32/979591 [..............................] - ETA: 130:38:17 - loss: 0.7055

InternalError: cuDNN Backward Filter function launch failure : input shape([32,300,1,70]) filter shape([1,3,300,100])
	 [[{{node training_1/Adam/gradients/conv1d_2/convolution/Conv2D_grad/Conv2DBackpropFilter}} = Conv2DBackpropFilter[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_1/Adam/gradients/conv1d_2/convolution/Conv2D_grad/Conv2DBackpropFilter-0-TransposeNHWCToNCHW-LayoutOptimizer, ConstantFolding/training_1/Adam/gradients/conv1d_2/convolution/Conv2D_grad/ShapeN-matshapes-1, training_1/Adam/gradients/conv1d_2/convolution/Conv2D_grad/Conv2DBackpropFilter-2-TransposeNHWCToNCHW-LayoutOptimizer)]]

In [23]:
cnn_model = create_cnn()
cnn_model.fit(train_seq_x, train_y)
predictions = cnn_model.predict(valid_seq_x, verbose=1)
predictions = predictions.argmax(axis=-1)

InternalError: GPU sync failed

In [ ]:
n = sub.columns[1]
sub.drop(n, axis = 1, inplace = True)
sub[n] = predictions
sub.to_csv("submission.csv", index=False)